In [13]:
import fiona
import geopandas as pd
import rasterio
import os

In [1]:
import os
import rasterio
from shapely.geometry import box
from rasterio.plot import show
from rasterio.plot import show_hist
from rasterio.mask import mask
import geopandas as gpd
from fiona.crs import from_epsg
import pycrs

#shp_path = '/home/qweddww123/farsite/TrainDataGen/WildfireShapefiles'
#lcpfilepath = '/home/qweddww123/farsite/TrainDataGen/GetLCPdata/LCP/lcp_extract'

shp_path = '/home/qweddww123/farsite/TrainDataGen/WildfireShapefiles/2016/al_caney_head/20160321_0000/shptest.shp'
fp = '/home/qweddww123/farsite/TrainDataGen/GetLCPdata/LCP/lcp_extract/2016/al_caney_head/al_caney_head.lcp'
out_lcp = '/home/qweddww123/farsite/TrainDataGen/GetLCPdata/LCP/lcp_extract/2016/al_caney_head/'+"cliptest1.lcp"

os.system('rm /home/qweddww123/farsite/TrainDataGen/GetLCPdata/LCP/lcp_extract/2016/al_caney_head/'+"cliptest1.lcp")

data = rasterio.open(fp)
#show((data, 4), cmap='terrain')

shpfile = gpd.read_file(shp_path)
x1 = float(shpfile.bounds.minx[0])
y1 = float(shpfile.bounds.miny[0])
x2 = float(shpfile.bounds.maxx[0])
y2 = float(shpfile.bounds.maxy[0])

minx, miny = x1, y1
maxx, maxy = x2, y2

bbox = box(minx, miny, maxx, maxy)

geo = gpd.GeoDataFrame({'geometry': bbox}, index=[0], crs=from_epsg(4326))
geo = geo.to_crs(crs="EPSG:4326")

def getFeatures(gdf):
    """Function to parse features from GeoDataFrame in such a manner that rasterio wants them"""
    import json
    return [json.loads(gdf.to_json())['features'][0]['geometry']]

coords = getFeatures(geo)
#print(coords)

out_img, out_transform = mask(data, shapes=coords, crop=True)

out_meta = data.meta.copy()
#print(out_meta)

epsg_code = 4326

out_meta.update({"driver": "GTiff",
                 "height": out_img.shape[1],
                 "width": out_img.shape[2],
                 "transform": out_transform,
                 "crs": pycrs.parse.from_epsg_code(epsg_code).to_proj4()}
               )

with rasterio.open(out_lcp, "w", **out_meta) as dest:
    dest.write(out_img)

clipped = rasterio.open(out_lcp)
#show((clipped, 4), cmap='terrain')

/home/qweddww123/anaconda3/envs/pygdal/lib/python3.9/site-packages/pyproj/crs/crs.py:131: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)
/home/qweddww123/anaconda3/envs/pygdal/lib/python3.9/site-packages/pyproj/crs/crs.py:131: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)
